## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Mataura,NZ,-46.1927,168.8643,46.71,98,31,0.65,scattered clouds
1,1,Dingle,PH,10.9995,122.6711,75.76,91,78,7.36,light rain
2,2,Jamestown,US,42.0970,-79.2353,54.84,81,1,3.00,clear sky
3,3,Albany,US,42.6001,-73.9662,52.48,80,73,3.13,broken clouds
4,4,Buenavista,PH,8.9769,125.4089,75.65,92,73,4.05,broken clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Dingle,PH,10.9995,122.6711,75.76,91,78,7.36,light rain
4,4,Buenavista,PH,8.9769,125.4089,75.65,92,73,4.05,broken clouds
7,7,Buluang,PH,13.3068,123.3425,75.76,91,98,4.07,overcast clouds
12,12,Vanimo,PG,-2.6741,141.3028,77.92,89,100,3.69,overcast clouds
14,14,San Patricio,US,28.0170,-97.5169,77.88,20,0,18.63,clear sky
25,25,Qui Nhon,VN,13.7667,109.2333,77.32,88,99,6.13,overcast clouds
27,27,Kapaa,US,22.0752,-159.3190,75.18,99,90,11.50,light rain
31,31,Nouadhibou,MR,20.9310,-17.0347,76.98,57,0,13.80,clear sky
33,33,Georgetown,MY,5.4112,100.3354,80.53,92,40,1.01,thunderstorm with light rain
36,36,Ocos,GT,14.5094,-92.1933,89.73,76,27,4.68,scattered clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                211
City                   211
Country                211
Lat                    211
Lng                    211
Max Temp               211
Humidity               211
Cloudiness             211
Wind Speed             211
Current Description    211
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
# ALL ROWS SHOW A COUNT OF 211, SO THERE ARE NO EMPTY ROWS.  
# WE WILL USE THE FOLLOWING ANYWAYS
clean_df = preferred_cities_df.dropna()

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Dingle,PH,75.76,light rain,10.9995,122.6711,
4,Buenavista,PH,75.65,broken clouds,8.9769,125.4089,
7,Buluang,PH,75.76,overcast clouds,13.3068,123.3425,
12,Vanimo,PG,77.92,overcast clouds,-2.6741,141.3028,
14,San Patricio,US,77.88,clear sky,28.0170,-97.5169,
25,Qui Nhon,VN,77.32,overcast clouds,13.7667,109.2333,
27,Kapaa,US,75.18,light rain,22.0752,-159.3190,
31,Nouadhibou,MR,76.98,clear sky,20.9310,-17.0347,
33,Georgetown,MY,80.53,thunderstorm with light rain,5.4112,100.3354,
36,Ocos,GT,89.73,scattered clouds,14.5094,-92.1933,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [9]:
hotel_df.head(20)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Dingle,PH,75.76,light rain,10.9995,122.6711,SEGAYA PROPERTY
4,Buenavista,PH,75.65,broken clouds,8.9769,125.4089,Room For Rent
7,Buluang,PH,75.76,overcast clouds,13.3068,123.3425,Enon Beach Resort
12,Vanimo,PG,77.92,overcast clouds,-2.6741,141.3028,Vahmoneh Lodge Vanimo
14,San Patricio,US,77.88,clear sky,28.0170,-97.5169,"Motel 6 Sinton, TX"
25,Qui Nhon,VN,77.32,overcast clouds,13.7667,109.2333,Hoang Yen Canary Hotel
27,Kapaa,US,75.18,light rain,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort
31,Nouadhibou,MR,76.98,clear sky,20.9310,-17.0347,El Medina
33,Georgetown,MY,80.53,thunderstorm with light rain,5.4112,100.3354,Cititel Penang
36,Ocos,GT,89.73,scattered clouds,14.5094,-92.1933,Auto Hotel El Soñador


In [10]:
# 7. Drop the rows where there is no Hotel Name.
# Import numpy
import numpy as np
# Replace empty values with NaN
hotel_df.replace('', np.nan, inplace=True)
# Drop NaN
clean_hotel_df = hotel_df.dropna()
clean_hotel_df.head(20)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Dingle,PH,75.76,light rain,10.9995,122.6711,SEGAYA PROPERTY
4,Buenavista,PH,75.65,broken clouds,8.9769,125.4089,Room For Rent
7,Buluang,PH,75.76,overcast clouds,13.3068,123.3425,Enon Beach Resort
12,Vanimo,PG,77.92,overcast clouds,-2.6741,141.3028,Vahmoneh Lodge Vanimo
14,San Patricio,US,77.88,clear sky,28.0170,-97.5169,"Motel 6 Sinton, TX"
25,Qui Nhon,VN,77.32,overcast clouds,13.7667,109.2333,Hoang Yen Canary Hotel
27,Kapaa,US,75.18,light rain,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort
31,Nouadhibou,MR,76.98,clear sky,20.9310,-17.0347,El Medina
33,Georgetown,MY,80.53,thunderstorm with light rain,5.4112,100.3354,Cititel Penang
36,Ocos,GT,89.73,scattered clouds,14.5094,-92.1933,Auto Hotel El Soñador


In [15]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [16]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [17]:
# 11a. Add a marker layer for each city to the map. 
max_temp = clean_hotel_df["Max Temp"]
fig = gmaps.figure(center=(40.0, -60.0), zoom_level=3.2)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# # 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))